# Logistic Regression Assignment Solution - Penguins

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
sns.set_theme(style="ticks")

import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

import warnings
warnings.simplefilter('ignore')

In [ ]:
penguins = sns.load_dataset('penguins')
penguins

In [ ]:
penguins.info()

### Can see the data has 
- Three categorical variables of type object: species, island, and sex
- Four continuous variabls of type float: bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g

There are a few nulls there. Let's just drop the nulls.

In [ ]:
penguins = penguins.dropna()
penguins.info()

### Draw some plots to investigate the data 

In [ ]:
sns.countplot(penguins['species']); #countplot is for getting counts on categorical categories

In [ ]:
# Note: not all species are on all islands
sns.countplot(x='island', hue='species', palette='Set2', data=penguins); #palette changes the color scheme

In [ ]:
sns.countplot(x='island', hue='sex', data=penguins); #hue gives a way to differentiate via color on a second categorical category

In [ ]:
# We will use this groupby to create a graph of sex by species and by island
penguins.groupby(['sex', 'species', 'island']).size().reset_index()

In [ ]:
sex = penguins.groupby(['sex', 'species', 'island']).size().reset_index().pivot(columns=['sex'], index=['island', 'species'], values=0)
sex

In [ ]:
# male and female peguin numbers are mostly balanced, except a few less female GENTOO penguins on Biscoe
#note we have used the pd.DataFrame.plot.barh() method here to plot the horizontal bars shown. There are many accceptable alternatives.
sex.plot.barh(figsize=(10,8)); 

In [ ]:
# The Gentoos are the heaviest species. They are only found on Biscoe Island.
_ , ax = plt.subplots(1,1,figsize=(12,8)) 
sns.boxplot(x='island', y='body_mass_g', hue='species', ax=ax, data=penguins);

### Now let's investigate the continuous data further

In [ ]:
penguins.describe()

In [ ]:
sns.boxplot( 'sex', 'flipper_length_mm', data=penguins);

In [ ]:
sns.boxplot( 'island', 'flipper_length_mm', hue='sex',data=penguins);

In [ ]:
sns.boxplot( 'sex', 'flipper_length_mm', hue='species', data=penguins);

In [ ]:
sns.boxplot( 'sex', 'body_mass_g', hue='island', data=penguins);

In [ ]:
sns.boxplot( 'sex', 'body_mass_g', hue='species', data=penguins);

We have three islands, and three species. 
However, our logistic regression model `smf.logit` only works on BINARY variables.
So, we need to decide exactly what we want to model.
Since sex is the only binary variable in this data, we'll use it in our model.

In [ ]:
print(penguins['sex'].value_counts()) #sex is coded as an object aka str type

In [ ]:
formula_1 = 'sex ~ C(species) + C(island) + bill_length_mm + bill_depth_mm + flipper_length_mm + body_mass_g'

In [ ]:
model_1 = smf.logit(formula=formula_1, data=penguins).fit() #this fails with a ValueError Exception
model_1.summary()

Statsmodels cannot deal with objects on the LHS. It only likes numbers there.
```
ValueError: endog has evaluated to an array with multiple columns that has shape (333, 2). This occurs when the variable converted to endog is non-numeric (e.g., bool or str)
```
So we need to create a new column to code sex as 0 or 1

Because we expect the males to be bigger on average, will code Male = 1 and Female = 0 (Try it the other way and see what changes!)

In [ ]:
penguins['male'] = np.where(penguins['sex']=='Male', 1,0)
penguins

In [ ]:
# Rewrite our model using penguins['male'] as the target

formula_2 = 'male ~ C(species) + C(island) + bill_length_mm + bill_depth_mm + flipper_length_mm + body_mass_g'
model_2 = smf.logit(formula=formula_2, data=penguins).fit()

model_2.summary()

We see that flipper_length_mm is not significant (p value = 0.581) given all the other measurement data in the above model. 
So let's drop flipper_length_mm and make a new model

In [ ]:
formula_3 = 'male ~ C(species) + C(island) + bill_length_mm + bill_depth_mm + body_mass_g'
model_3 = smf.logit(formula=formula_3, data=penguins).fit()
model_3.summary()

We also see that **neither** of the coefficients of `island` is significant (p=0.662 and p=0.610). 

In this data, once we know the species, the island is not redundant for predicting sex. So let's drop `island` as well.

Aside: If even one coefficient of a categorical variable is significant, we should keep it in the model.

In [ ]:
formula_4 = 'male ~ C(species) + bill_length_mm + bill_depth_mm + body_mass_g'
model_4 = smf.logit(formula=formula_4, data=penguins).fit()
model_4.summary()

So, since all coefficients are now significant, we're done.
Let's use model_2, model_3, and model_4 for prediction metrics like our confusion_matrix, precision_score, and recall_score

In [ ]:
# these are all the predicted probability of male for each penguin for each model
for model in [model_2, model_3, model_4]:
    print(model.predict(penguins))

Let's say that if the prediction probability > 0.5, the model predicts male

In [ ]:
# compute predictions for model 2
y_prediction_2 = np.where(model_2.predict(penguins) >= 0.5,1,0)
y_prediction_2

In [ ]:
# Do the same for model_3 and model_4
y_prediction_3 = np.where(model_3.predict(penguins) >= 0.5,1,0)
y_prediction_4 = np.where(model_4.predict(penguins) >= 0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
# Now, let's compute the confusion matrices for each model

confusion_matrix_2 = confusion_matrix(penguins['male'], y_prediction_2 )
confusion_matrix_3 = confusion_matrix(penguins['male'], y_prediction_3 )
confusion_matrix_4 = confusion_matrix(penguins['male'], y_prediction_4 )

In [ ]:
# Now, let's compute the precision and recall scores for each model
recall_score_2 = recall_score(penguins['male'], y_prediction_2 )
recall_score_3 = recall_score(penguins['male'], y_prediction_3 )
recall_score_4 = recall_score(penguins['male'], y_prediction_4 )
precision_score_2 = precision_score(penguins['male'], y_prediction_2 )
precision_score_3 = precision_score(penguins['male'], y_prediction_3 )
precision_score_4 = precision_score(penguins['male'], y_prediction_4 )

In [ ]:
print(formula_2)
print(confusion_matrix_2)
print('recall', recall_score_2)
print('precision', precision_score_2)

In [ ]:
print(formula_3)
print(confusion_matrix_3)
print('recall', recall_score_3)
print('precision', precision_score_3)

In [ ]:
print(formula_4)
print(confusion_matrix_4)
print('recall', recall_score_4)
print('precision', precision_score_4)

By definition a confusion matrix `C[i, j]`
    is equal to the number of observations known to be in group `i` and
    predicted by the model to be in group `j`.

Note: We are testing these models on data that the model was built on. 
    So it is likely that these models would perform worse on previously unseen data!

References
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics